In [15]:
import string
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel

In [2]:
df = pd.read_csv('./playlist_data.csv')

In [3]:
df

,playlist_id,playlist,genre
0,37i9dQZF1DXcBWIGoYBM5M,"['Heat Waves Glass Animals', 'Ghost Justin Bie...",pop
1,37i9dQZF1DX0kbJZpiYdZl,"['Big Energy Latto', 'Ghost Justin Bieber', 'H...",pop
2,37i9dQZF1DX4JAvHpjipBk,"['CHICKEN TERIYAKI ROSALÍA', 'Usain Boo Kodak ...",pop
3,37i9dQZF1DWUa8ZRTfalHk,"['Go Cat Burns', ""she's all i wanna be Tate Mc...",pop
4,37i9dQZF1DX5Vy6DFOcx00,"['Mount Everest Labrinth', 'MIDDLE OF THE NIGH...",pop
...,...,...,...
195,37i9dQZF1DX6RbMTl2YnQ8,"['Vía Láctea Zoé', 'Irresponsables Babasónicos...",rock
196,37i9dQZF1DXbFRZSqP41al,"['Hash Pipe Weezer', 'Celebrity Skin Hole', 'A...",rock
197,37i9dQZF1DX9E1mLvTvD1f,['Bullet With Butterfly Wings - Remastered 201...,rock
198,37i9dQZF1DX8V6C5Nr1MvC,['Good Riddance (Time of Your Life) Green Day'...,rock


In [4]:
playlists = [re.sub('[^a-zA-Z0-9 \n\.]', '', i) for i in df.playlist]

In [5]:
playlists[0]

'Heat Waves Glass Animals Ghost Justin Bieber MAMIII Becky G Shivers Ed Sheeran Enemy with JID  from the series Arcane League of Legends Imagine Dragons THATS WHAT I WANT Lil Nas X Infinity Jaymes Young STAY with Justin Bieber The Kid LAROI abcdefu GAYLE Cold Heart  PNAU Remix Elton John Woman Doja Cat INDUSTRY BABY feat. Jack Harlow Lil Nas X Light Switch Charlie Puth Boyfriend Dove Cameron Easy On Me Adele Where Are You Now Lost Frequencies love nwantiti ah ah ah CKay shes all i wanna be Tate McRae Bad Habits Ed Sheeran My Universe Coldplay Oh My God Adele TO THE MOON Jnr Choi Desesperados Rauw Alejandro Need to Know Doja Cat One Right Now with The Weeknd Post Malone Nail Tech Jack Harlow pushin P feat. Young Thug Gunna The Motto Tisto Pepas Farruko MIDDLE OF THE NIGHT Elley Duh I Love You So The Walters Happier Than Ever  Edit Billie Eilish CHICKEN TERIYAKI ROSALA Super Gremlin Kodak Black Sacrifice The Weeknd Fingers Crossed Lauren SpencerSmith The Joker And The Queen feat. Taylor 

In [13]:
df['clean'] = playlists

In [14]:
df

,playlist_id,playlist,genre,clean
0,37i9dQZF1DXcBWIGoYBM5M,"['Heat Waves Glass Animals', 'Ghost Justin Bie...",pop,Heat Waves Glass Animals Ghost Justin Bieber M...
1,37i9dQZF1DX0kbJZpiYdZl,"['Big Energy Latto', 'Ghost Justin Bieber', 'H...",pop,Big Energy Latto Ghost Justin Bieber Heat Wave...
2,37i9dQZF1DX4JAvHpjipBk,"['CHICKEN TERIYAKI ROSALÍA', 'Usain Boo Kodak ...",pop,CHICKEN TERIYAKI ROSALA Usain Boo Kodak Black ...
3,37i9dQZF1DWUa8ZRTfalHk,"['Go Cat Burns', ""she's all i wanna be Tate Mc...",pop,Go Cat Burns shes all i wanna be Tate McRae We...
4,37i9dQZF1DX5Vy6DFOcx00,"['Mount Everest Labrinth', 'MIDDLE OF THE NIGH...",pop,Mount Everest Labrinth MIDDLE OF THE NIGHT Ell...
...,...,...,...,...
195,37i9dQZF1DX6RbMTl2YnQ8,"['Vía Láctea Zoé', 'Irresponsables Babasónicos...",rock,Va Lctea Zo Irresponsables Babasnicos Tornasol...
196,37i9dQZF1DXbFRZSqP41al,"['Hash Pipe Weezer', 'Celebrity Skin Hole', 'A...",rock,Hash Pipe Weezer Celebrity Skin Hole Are You G...
197,37i9dQZF1DX9E1mLvTvD1f,['Bullet With Butterfly Wings - Remastered 201...,rock,Bullet With Butterfly Wings Remastered 2012 T...
198,37i9dQZF1DX8V6C5Nr1MvC,['Good Riddance (Time of Your Life) Green Day'...,rock,Good Riddance Time of Your Life Green Day Nuts...


In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [17]:
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
encoded_input_pop = tokenizer(df.clean[0], return_tensors='pt')

In [20]:
output_pop = model(**encoded_input_pop)

In [37]:
output_pop.last_hidden_state[0]

tensor([[-0.5438, -0.4138,  0.3156,  ..., -0.1353, -0.0162, -0.0314],
        [ 0.3200, -0.1504,  0.9230,  ..., -0.2143,  0.2521, -0.1553],
        [ 0.8855,  0.3419,  0.3564,  ..., -0.0205, -0.3665,  0.1655],
        ...,
        [-0.4896, -0.1427,  0.8944,  ...,  0.1764,  0.2645, -0.7269],
        [-0.4164, -0.3330,  0.4605,  ..., -0.0401, -0.5874, -1.1509],
        [ 0.4611,  0.0940, -0.0270,  ...,  0.0462, -0.4190, -0.1923]],
       grad_fn=<SelectBackward0>)

In [23]:
encoded_input_rock = tokenizer(df.clean[196], return_tensors='pt')

In [24]:
output_rock = model(**encoded_input_rock)

In [38]:
output_rock.last_hidden_state[0]

tensor([[-0.3359,  0.0838,  0.0361,  ..., -0.2519, -0.1168,  0.0579],
        [ 1.1298,  0.0732,  1.2148,  ...,  0.4544,  0.2317,  0.1134],
        [ 1.2555,  0.6489,  0.1931,  ...,  0.5193,  0.0042,  0.1893],
        ...,
        [-0.1697, -0.2646,  0.3771,  ...,  0.2946,  0.5077, -0.6802],
        [ 0.0254, -0.5127, -0.2497,  ..., -0.0356, -0.2451, -0.6891],
        [ 0.3044,  0.3101,  0.2565,  ..., -0.0029, -0.1546, -0.2339]],
       grad_fn=<SelectBackward0>)

In [43]:
cosine_similarity(output_rock.last_hidden_state[0].detach().numpy(), output_pop.last_hidden_state[0].detach().numpy())

array([[ 0.867285  ,  0.18579535,  0.17692077, ...,  0.09168494,
         0.10935813, -0.00762907],
       [ 0.08839131,  0.43492937,  0.24064448, ...,  0.19312033,
         0.13455278, -0.00212963],
       [ 0.11667665,  0.38078865,  0.45317   , ...,  0.12046961,
         0.19445944,  0.01103076],
       ...,
       [ 0.07175101,  0.32243925,  0.17598103, ...,  0.1531674 ,
         0.13981408,  0.03072366],
       [ 0.10651955,  0.30066708,  0.2728317 , ...,  0.18985656,
         0.29397225,  0.03085668],
       [ 0.04705387,  0.03022932,  0.14808308, ...,  0.01453922,
         0.06329374,  0.89476836]], dtype=float32)

In [54]:
encoded_playlists = [tokenizer(i, return_tensors='pt') for i in df.clean]

In [55]:
outputs = [model(**i) for i in encoded_playlists]

RuntimeError: The size of tensor a (651) must match the size of tensor b (512) at non-singleton dimension 1